# xihr Simulation Demo

This notebook demonstrates how to run the event-driven simulation engine against the bundled sample dataset using the naive favourite strategy.


## Prerequisites

Ensure the project dependencies are installed in your environment. If you are working in a clean virtual environment, install the package in editable mode with:

```bash
pip install -e .
```

The sample data used below lives in `data/sample`.


In [ ]:
from dataclasses import asdict
from datetime import timedelta
from pathlib import Path

import pandas as pd

from xihr.adaptors import CSVDataAdaptor
from xihr.analytics import generate_report
from xihr.engine import Engine
from xihr.portfolio import Portfolio
from xihr.repositories import SimulationBettingRepository, SimulationDataRepository
from strategies import NaiveFavoriteStrategy


## Load data and configure the engine

We will backtest the naive favourite strategy with the sample CSV dataset.


In [ ]:
data_path = Path('..') / 'data' / 'sample'
adaptor = CSVDataAdaptor(data_path)
data_repository = SimulationDataRepository(adaptor, payoff_publication_delay=timedelta(minutes=30))
portfolio = Portfolio.create(1000.0)
betting_repository = SimulationBettingRepository(portfolio, data_repository)
engine = Engine(data_repository, betting_repository)
strategy = NaiveFavoriteStrategy()
engine.run(strategy)
positions = betting_repository.get_positions()
print(f'Placed {len(positions)} bets; ending bankroll: {portfolio.bankroll():.2f}')


## Inspect the generated bets

The portfolio records each bet that was confirmed by the simulated broker.


In [ ]:
positions_df = pd.DataFrame([
    {
        'bet_id': pos.bet_id,
        'race_id': pos.race_id,
        'bet_type': pos.bet_type,
        'combination': '-'.join(pos.combination),
        'stake': pos.stake,
        'payout': pos.payout,
        'status': pos.status,
        'placed_at': pos.placed_at.isoformat(),
    }
    for pos in positions
])
positions_df.sort_values('race_id').reset_index(drop=True)


## Summarise performance

Finally, we compute some key performance indicators for the completed run.


In [ ]:
report = generate_report(positions)
pd.DataFrame([asdict(report)])
